# Import libraries

In [ ]:
%pylab inline

In [ ]:
import seaborn as sns

In [ ]:
import pandas as pd

# Read the data

Data from the **From Organized High-Throughput Data to Phenomenological Theory using Machine Learning: The Example of Dielectric Breakdown**  (https://pubs.acs.org/doi/10.1021/acs.chemmater.5b04109)

In [ ]:
df=pd.read_csv("data.txt", sep="\s+")
df

In [ ]:
df.info()

In [ ]:
df.describe()

## One-hot encoding of the categorical variable

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df.Structure.unique()

In [ ]:
ohe = OneHotEncoder()

In [ ]:
structure_ohe=ohe.fit_transform(df["Structure"].values.reshape(-1,1))

In [ ]:
structure_ohe=structure_ohe.todense()

In [ ]:
ohe.categories_[0]

In [ ]:
ohe_df=pd.DataFrame(structure_ohe,columns=ohe.categories_[0])
ohe_df

In [ ]:
tot_df=pd.concat([df,ohe_df], axis=1)

In [ ]:
tot_df

In [ ]:
tot_df.columns

Selection of the columns for training

In [ ]:
X = tot_df[['Eg', 'omega_max', 'omega_mean', 'epsilon_e', 'epsilon_tot', 'dNN', 'rho', 'M',  'CC', 'RS','ZB']]
#X = tot_df[['Eg', 'omega_max', 'omega_mean', 'epsilon_e', 'epsilon_tot', 'dNN', 'rho', 'M']]

In [ ]:
y = tot_df['Fb']

In [ ]:
X.shape

In [ ]:
y.shape

# Perform a standard scaling of the features for better convergence

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std_scaler = StandardScaler()

In [ ]:
X_std = pd.DataFrame(std_scaler.fit_transform(X), columns = X.columns)

In [ ]:
X_std.head()

In [ ]:
plt.hist(y, bins=20)

Target variable is skewed. Let us perform a log(1+y) transformation of it with `np.log1p` function

In [ ]:
yln = np.log1p(y)

In [ ]:
plt.hist(yln, bins=20)

Show an overall pariplot

In [ ]:
sns.pairplot(tot_df)

# Validation scheme

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

for the train/test split validation

In [ ]:
X_train, X_test, X_std_train, X_std_test, y_train, y_test, yln_train, yln_test = train_test_split(
    X, X_std, y, yln, test_size=0.15, random_state=42)

for the cross-validation

In [ ]:
cv = KFold(n_splits=10,shuffle=True, random_state=42)

define new metric function - Mean Absolute Relative Error

In [ ]:
def MARE(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred)/np.abs(y_true))

In [ ]:
def my_scorer_cv(scorer, X_=X_std, y_=y, exponentiate=False):
    if exponentiate:
        y_orig = y_
        y_ln = np.log1p(y_orig)

        y_pred_ln = cross_val_predict(scorer, X_, y_ln, cv=cv)

        y_pred = np.expm1(y_pred_ln)
        print("R2 = {},\nMARE={},\nMAE = {},\nRMSE={}".format(r2_score(y_orig, y_pred), MARE(
            y_orig, y_pred), mean_absolute_error(y_orig, y_pred), np.sqrt(mean_squared_error(y_orig, y_pred))))

        y_max = max(max(y_orig), max(y_pred))
        y_min = min(min(y_orig), min(y_pred))

        plt.xlim(y_min, y_max)
        plt.ylim(y_min, y_max)
        plt.scatter(y_orig, y_pred, alpha=0.75)
        plt.plot([y_min, y_max], [y_min, y_max])
        plt.xlabel("True F$_b$")
        plt.ylabel("Predicted F$_b$")
        plt.show()
        plt.scatter(y_ln, y_pred_ln, alpha=0.75)
        plt.xlabel("True $ \ln$ F$_b$")
        plt.ylabel("Predicted $\ln$ F$_b$")
        y_max = max(max(y_ln), max(y_pred_ln))
        y_min = min(min(y_ln), min(y_pred_ln))
        plt.plot([y_min, y_max], [y_min, y_max])
        plt.xlim(y_min, y_max)
        plt.ylim(y_min, y_max)
        return

    y_pred = cross_val_predict(scorer, X_, y_, cv=cv)
    print("R2 = {},\nMARE={},\nMAE = {},\nRMSE={}".format(r2_score(y, y_pred), MARE(
        y, y_pred), mean_absolute_error(y, y_pred), np.sqrt(mean_squared_error(y, y_pred))))
    y_max = max(max(y), max(y_pred))
    y_min = min(min(y), min(y_pred))
    plt.xlim(y_min, y_max)
    plt.ylim(y_min, y_max)
    plt.scatter(y_, y_pred, alpha=0.75)
    plt.plot([y_min, y_max], [y_min, y_max])
    plt.xlabel("True F$_b$")
    plt.ylabel("Predicted F$_b$")
    plt.show()

In [ ]:
def my_scorer_test(scorer, X_train=X_std_train, X_test=X_std_test, y_train=y_train, y_test=y_test, exponentiate=False):
    y_orig_train, y_orig_test = y_train, y_test
    if exponentiate:

        y_ln_train = np.log1p(y_orig_train)
        y_ln_test = np.log1p(y_orig_test)

        scorer.fit(X_train, y_ln_train)
        y_ln_train_pred = scorer.predict(X_train)
        y_ln_test_pred = scorer.predict(X_test)

        y_pred_train = np.expm1(y_ln_train_pred)
        y_pred_test = np.expm1(y_ln_test_pred)
        print("TEST:\nR2 = {},\nMARE={},\nMAE = {},\nRMSE={}".format(
            r2_score(y_orig_test, y_pred_test),
            MARE(y_orig_test, y_pred_test),
            mean_absolute_error(y_orig_test, y_pred_test),
            np.sqrt(mean_squared_error(y_orig_test, y_pred_test)))
        )
        print("TRAIN:\nR2 = {},\nMARE={},\nMAE = {},\nRMSE={}".format(
            r2_score(y_orig_train, y_pred_train),
            MARE(y_orig_train, y_pred_train),
            mean_absolute_error(y_orig_train, y_pred_train),
            np.sqrt(mean_squared_error(y_orig_train, y_pred_train)))
        )

        y_max = max(list(y_orig_test) + list(y_orig_train) +
                    list(y_pred_test) + list(y_pred_train))
        y_min = min(list(y_orig_test) + list(y_orig_train) +
                    list(y_pred_test) + list(y_pred_train))

        plt.xlim(y_min, y_max)
        plt.ylim(y_min, y_max)
        plt.scatter(y_orig_train, y_pred_train, alpha=0.75,
                    color="blue", label="Train")
        plt.scatter(y_orig_test, y_pred_test, alpha=0.75,
                    color="green", label="Test")
        plt.plot([y_min, y_max], [y_min, y_max])
        plt.xlabel("True F$_b$")
        plt.ylabel("Predicted F$_b$")
        plt.legend()
        plt.show()

        plt.scatter(y_ln_train, y_ln_train_pred,
                    alpha=0.75, color="blue", label="Train")
        plt.scatter(y_ln_test, y_ln_test_pred, alpha=0.75,
                    color="green", label="Test")
        plt.xlabel("True $ \ln$ F$_b$")
        plt.ylabel("Predicted $\ln$ F$_b$")
        plt.legend()
        y_max = max(list(y_ln_train) + list(y_ln_train_pred) +
                    list(y_ln_test) + list(y_ln_test_pred))
        y_min = min(list(y_ln_train) + list(y_ln_train_pred) +
                    list(y_ln_test) + list(y_ln_test_pred))
        plt.plot([y_min, y_max], [y_min, y_max])
        plt.xlim(y_min, y_max)
        plt.ylim(y_min, y_max)
        return

    scorer.fit(X_train, y_train)
    y_pred_train = scorer.predict(X_train)
    y_pred_test = scorer.predict(X_test)

    print("TEST:\nR2 = {},\nMARE={},\nMAE = {},\nRMSE={}".format(
        r2_score(y_orig_test, y_pred_test),
        MARE(y_orig_test, y_pred_test),
        mean_absolute_error(y_orig_test, y_pred_test),
        np.sqrt(mean_squared_error(y_orig_test, y_pred_test)))
    )

    print("TRAIN:\nR2 = {},\nMARE={},\nMAE = {},\nRMSE={}".format(
        r2_score(y_orig_train, y_pred_train),
        MARE(y_orig_train, y_pred_train),
        mean_absolute_error(y_orig_train, y_pred_train),
        np.sqrt(mean_squared_error(y_orig_train, y_pred_train)))
    )

    y_max = max(list(y_orig_test) + list(y_orig_train) +
                list(y_pred_test) + list(y_pred_train))
    y_min = min(list(y_orig_test) + list(y_orig_train) +
                list(y_pred_test) + list(y_pred_train))

    plt.xlim(y_min, y_max)
    plt.ylim(y_min, y_max)
    plt.scatter(y_orig_train, y_pred_train, alpha=0.75,
                color="blue", label="Train")
    plt.scatter(y_orig_test, y_pred_test, alpha=0.75,
                color="green", label="Test")
    plt.plot([y_min, y_max], [y_min, y_max])
    plt.xlabel("True F$_b$")
    plt.ylabel("Predicted F$_b$")
    plt.legend()
    plt.show()

# Complex features model from the paper

The best model from supporting information, section S7: https://pubs.acs.org/doi/suppl/10.1021/acs.chemmater.5b04109/suppl_file/cm5b04109_si_001.pdf

$F_b = 43.554 \exp\{0.173 \omega_{max} E_g^{ 1/2} / \ln(\omega_{mean})\} $

compute for the whole dataset

In [ ]:
y_complex_pred = 43.554 * np.exp(0.173*df["omega_max"]*df["Eg"]**(1./2.)/ np.log(df["omega_mean"]))

compute for the test dataset only

In [ ]:
y_complex_test_pred = 43.554 * np.exp(0.173*X_test["omega_max"]*X_test["Eg"]**(1./2.)/ np.log(X_test["omega_mean"]))

In [ ]:
plt.scatter(y,y_complex_pred, label="Train")
plt.scatter(y_test,y_complex_test_pred, label="Test")
plt.legend()

Metrics on the whole dataset

In [ ]:
mean_absolute_error(y_complex_pred,y)

In [ ]:
np.sqrt(mean_squared_error(y,y_complex_pred))

In [ ]:
MARE(y,y_complex_pred)

In [ ]:
r2_score(y,y_complex_pred)

Metrics on the **test** dataset

In [ ]:
mean_absolute_error(y_complex_test_pred,y_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_complex_test_pred))

In [ ]:
MARE(y_test, y_complex_test_pred)

In [ ]:
r2_score(y_test, y_complex_test_pred)

# Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

In [ ]:
my_scorer_cv(lr)

In [ ]:
my_scorer_test(lr)

Try the logarithm and inverse-exponentiation transformation

In [ ]:
my_scorer_test(lr, exponentiate=True)

In [ ]:
my_scorer_cv(lr, exponentiate=True)

# Kernel Ridge

In [ ]:
from sklearn.kernel_ridge import KernelRidge

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Neural network (sklearn)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp=MLPRegressor(random_state=42,max_iter=2000)

# Neural network - Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, Adadelta
from keras.callbacks import EarlyStopping

In [ ]:
model=Sequential()
model.add(Dense(10,input_shape=(X_std.shape[1],), activation="relu"))
model.add(Dense(5, activation="relu"))
model.add(Dense(1, activation="linear"))

In [ ]:
model.compile(optimizer=Adam(lr=0.005),loss="mse",)

In [ ]:
model.summary()

In [ ]:
early_stopping = EarlyStopping(patience=15)

In [ ]:
model.fit(X_std_train,np.log1p(y_train), epochs=2000, validation_data=(X_std_test, np.log1p(y_test)), verbose=2,
         callbacks=[early_stopping])

In [ ]:
yln_test_pred = model.predict(X_std_test)
yln_train_pred = model.predict(X_std_train)

In [ ]:
y_pred_test=np.expm1(yln_test_pred).reshape(-1)
y_pred_train=np.expm1(yln_train_pred).reshape(-1)

In [ ]:
plt.scatter(y_train,y_pred_train,label="Train")
plt.scatter(y_test,y_pred_test, label="Test")
plt.legend()

In [ ]:
r2_score(y_train,y_pred_train)

In [ ]:
r2_score(y_test,y_pred_test)

In [ ]:
mean_absolute_error(y_test,y_pred_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred_test))